# Banco de dados Solo Criado: Take 1

Vou fazer primeiro no notebook para ir testando, depois passo para o arquivo .py

## Roadmap

* Coletar listagem do site da prefeitura
* Transformar pdf da listagem em xmls por este site
* Transformar xmls em cvs
* Carregar cvs de listagem para o banco de dados
  * Extração;
  * Exploração;
  * Limpeza;
  * Agregação;
  * Visualização;
  * Storytelling.

## Primeira parte: Coleta e transformação dos dados

from [here](https://www.edureka.co/blog/web-scraping-with-python/): <br>
Libraries used for Web Scraping  <br>
As we know, Python is has various applications and there are different libraries for different purposes. In our further demonstration, we will be using the following libraries: <br>
<br>
**Selenium**:  Selenium is a web testing library. It is used to automate browser activities. <br>
**BeautifulSoup**: Beautiful Soup is a Python package for parsing HTML and XML documents. It creates parse trees that is helpful to extract the data easily.<br>
Pandas: Pandas is a library used for data manipulation and analysis. It is used to extract the data and store it in the desired format. 

Eu tenho que baixar com requests somente a parte que me interessa (div id="conteudo" class="outros") <br>
Ou usar o bs4 pra retirar somente essa parte do todo

O código abaixo recebe e cria um arquivo html com os dados da página de listagens.

In [1]:
#import requests
#
#url = 'http://www2.portoalegre.rs.gov.br/edificapoa/default.php?p_secao=1445'
#r = requests.get(url)
#
#open('estoques.html', 'wb').write(r.content)

O código abaixo (em andamento) serve para limpar o html e eventualmente retirar os links das listagens em pdf

In [2]:
#from bs4 import BeautifulSoup
#
#with open('estoques.html') as fp:
#    soup = BeautifulSoup(fp, 'html.parser')
#
#conteudo = soup.find(id="conteudo")
#conteudo.find_all('strong')
#conteudo.find_all('a')
#for link in conteudo.find_all('a'):
#    print(link.get('href'))

Ao longo do tempo o formato de cada entrada de listagem mudou, então o ideal seria criar uma lista com as listagens já existentes e ir adicionando com base na ultima listagem. O problema é se mudarem o formato de divulgação de novo. acho que vou focar somente na parte de banco de dados que á mais útil e confiável, por enquanto.

## Segunda parte: a criação do banco de dados

### 1. Extração

imports

In [3]:
import pandas as pd
import os
#import geopandas as gpd
import functools as ft

lista de arquivos

In [4]:
print(os.listdir("./dados/ListagemSoloCriado/XLSX"))

['estq-max-01.xlsx', 'estq-max-02.xlsx', 'estq-max-03.xlsx', 'estq-max-04.xlsx', 'estq-max-05.xlsx', 'estq-max-06.xlsx', 'estq-max-07.xlsx', 'estq-max-08.xlsx', 'estq-max-09.xlsx', 'estq-max-10.xlsx', 'estq-max-11.xlsx', 'estq-max-12.xlsx', 'estq-max-13.xlsx', 'estq-max-14.xlsx', 'estq-max-15.xlsx', 'estq-max-16.xlsx']


join_sheet

In [5]:
def join_sheet(file):
    """Receives a excel file path and
    returns a dataframe with his sheets
    appended. Also fix third column name."""

    sheet_list = pd.ExcelFile(file).sheet_names
    df_list = []
    for sheet in sheet_list:
        df_list.append(pd.read_excel(io=file, sheet_name=sheet, header=1))
    
    listagem = pd.concat(objs=df_list, ignore_index=True)
    
    colunas = listagem.columns.to_list()
    colunas[-1] = 'estq_max_' + file[-7:-5]
    listagem.columns = colunas

    return listagem


mzueuqrt

In [6]:
def mzueuqrt(dataframe: list):
    """Receives a dataframe, adds
    the MZUEUQRT column in the [-1]
    position, and makes it the index."""

    new_columns = list(dataframe.columns)
    new_columns.insert(-1, 'MZUEUQRT')
    dataframe = dataframe.reindex(columns=new_columns)
    #Agora vem o calculo da columa MZUEUQRT
    
    dataframe['MZUEUQRT'] = (dataframe['MZ']*1000000) + (dataframe['UEU']*1000) + (dataframe['QRT'])

    return dataframe

db_manager

In [7]:
def db_manager(path=any, database=[]):
    """Receives a path to Excel files and 
    a list containing dataframes, checks if 
    there is any file that is not in the database, 
    adds the missing files to database if there is any 
    and returns the updated database."""

    files = os.listdir(path)
    files.sort()
    
    if len(database) == len(files): #Não há nenhum dado novo a ser adicionado
        print("O banco de dados já está atualizado.")
        return database
    elif len(database) > len(files): #Há algum problema, pois o BD possui mais entradas do que a pasta de arquivos
        print("O banco de dados possui mais entradas do que a pasta de arquivos")
        return database
    elif len(database) < len(files): #Existem novas listagens que não foram adicionadas ao BD
        print("O banco de dados está desatualizado")
        for i in range(len(database), len(files)):
            listagem = mzueuqrt(join_sheet(path + "/" + files[i]))
            listagem = listagem.set_index('MZUEUQRT')
            database.append(listagem)
            print("Adicionando listagem", (i+1))

    return database
    

Testando a função db_manager

In [8]:
db_estq = db_manager("./dados/ListagemSoloCriado/XLSX")


O banco de dados está desatualizado
Adicionando listagem 1
Adicionando listagem 2
Adicionando listagem 3
Adicionando listagem 4
Adicionando listagem 5
Adicionando listagem 6
Adicionando listagem 7
Adicionando listagem 8
Adicionando listagem 9
Adicionando listagem 10
Adicionando listagem 11
Adicionando listagem 12
Adicionando listagem 13
Adicionando listagem 14
Adicionando listagem 15
Adicionando listagem 16


In [9]:
#estq_max_01 = db_estq[0]
#estq_max_02 = db_estq[1]
#estq_max_03 = db_estq[2]
#estq_max_04 = db_estq[3]
#estq_max_05 = db_estq[4]
#estq_max_06 = db_estq[5]
#estq_max_07 = db_estq[6]
#estq_max_08 = db_estq[7]
#estq_max_09 = db_estq[8]
#estq_max_10 = db_estq[9]
#estq_max_11 = db_estq[10]
#estq_max_12 = db_estq[11]
#estq_max_13 = db_estq[12]
#estq_max_14 = db_estq[13]
#estq_max_15 = db_estq[14]
#estq_max_16 = db_estq[15]

## Terceira parte: Manipulação do Banco de Dados

### Agregação

estoque_maximo

In [10]:
def estoque_maximo(database):
    """Receives a database containing 
    all dataframes, and aggregates them."""

    estq_max = database[0]

    for i in range(1, len(database)):
        if i < 9: #Somente para não ter erro com a coluna da direita        \/
            estq_max = pd.merge(left=estq_max, right=database[i][('estq_max_0'+str(i+1))], how='outer', on='MZUEUQRT')
        else:
            estq_max = pd.merge(left=estq_max, right=database[i][('estq_max_'+str(i+1))], how='outer', on='MZUEUQRT')
    
    return estq_max

testando a função estoque_maximo

In [11]:
estq_max_num = estoque_maximo(db_estq)
estq_max_num

,MZ,UEU,QRT,estq_max_01,estq_max_02,estq_max_03,estq_max_04,estq_max_05,estq_max_06,estq_max_07,estq_max_08,estq_max_09,estq_max_10,estq_max_11,estq_max_12,estq_max_13,estq_max_14,estq_max_15,estq_max_16
MZUEUQRT,,,,,,,,,,,,,,,,,,,
1002001,1.0,2.0,1.0,AE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002003,1.0,2.0,3.0,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE
1002005,1.0,2.0,5.0,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE
1004003,1.0,4.0,3.0,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE
1004005,1.0,4.0,5.0,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,2540.07,2540.07,2540.07,2540.07,2540.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3140091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AE,AE
3140093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7252.26,7252.26


In [12]:
#estq_max_num.index[estq_max_num['estq_max_01'] = = 'AE'].tolist()

In [13]:
#type(estq_max_num['estq_max_01'][1004005])
estq_max_num['estq_max_01']

MZUEUQRT
1002001           AE
1002003           AE
1002005           AE
1004003           AE
1004005    1388.5667
             ...    
3140089          NaN
3140091          NaN
3140093          NaN
3140095          NaN
3140097          NaN
Name: estq_max_01, Length: 6283, dtype: object

Função que separa as áreas especiais

retira_ae

In [14]:
def retira_ae(dataframe):
    colunas = list(dataframe.columns)

    for i in range(3, len(colunas)):
        area_especial = list(dataframe.index[dataframe[colunas[i]] == 'AE'])
        dataframe.drop(labels=area_especial, axis=0, inplace=True)
    
    return dataframe


retorna_ae

In [15]:
def retorna_ae(dataframe):
    colunas = list(dataframe.columns)

    for i in range(3, len(colunas)):
        area_especial = list(dataframe.index[dataframe[colunas[i]] != 'AE'])
        dataframe.drop(labels=area_especial, axis=0, inplace=True)
    
    return dataframe

In [16]:
#area_especial = list(estq_max_num.index[estq_max_num['estq_max_01'] == 'AE'])
area_especial = retira_ae(estq_max_num)
area_especial.head(5)

,MZ,UEU,QRT,estq_max_01,estq_max_02,estq_max_03,estq_max_04,estq_max_05,estq_max_06,estq_max_07,estq_max_08,estq_max_09,estq_max_10,estq_max_11,estq_max_12,estq_max_13,estq_max_14,estq_max_15,estq_max_16
MZUEUQRT,,,,,,,,,,,,,,,,,,,
1004005,1.0,4.0,5.0,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,2540.07,2540.07,2540.07,2540.07,2540.07
1004007,1.0,4.0,7.0,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,4327.49,4327.49,4327.49,4327.49,4327.49
1004009,1.0,4.0,9.0,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,3727.71,3727.71,3727.71,3727.71,3727.71
1004013,1.0,4.0,13.0,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,7794.62,7794.62,7794.62,7794.62,7794.62
1004015,1.0,4.0,15.0,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,9313.57,9313.57,9313.57,9313.57,9313.57


In [17]:
teste = pd.DataFrame({'a': [1, 2, 3], 'b': [2, 3, 4], 'c': [3, 4, 5]})

colunas = list(teste.columns)
teste[colunas[2]] = teste[colunas[0]] - teste[colunas[1]]
teste

,a,b,c
0,1,2,-1
1,2,3,-1
2,3,4,-1


In [18]:
rem = teste.index[teste['a'] == 2].tolist()
teste.drop(labels=rem, axis=0) #sucesso

,a,b,c
0,1,2,-1
2,3,4,-1


Agora uma função que cria o dataframe de uso de solo criado

In [20]:
#estq_max_num.to_csv(path_or_buf="./dados/estq_max_num.csv", sep=';') #já rodei o codigo 1 vez

In [41]:
uso_estq = estq_max_num.copy()

colunas = list(uso_estq.columns)

#colunas[3] #estq_max_01

for i in range(4, len(colunas)):
    uso_estq[colunas[i]] = (estq_max_num[colunas[i-1]] - estq_max_num[colunas[i]])
    #print(colunas[i])

In [44]:
def uso_estoque(database, is_percent=False): #arrumar
    """Receives a database containing 
    all dataframes, aggregates them, and
    calculate the use of Solo Criado."""
    if is_percent:
        percent = (100 / database[0])
        prct = '%'
    else:
        percent = 1
        prct = ''

    uso_estq = database[0].copy()
    uso_estq = retira_ae(uso_estq)
    colunas = list(uso_estq.columns)
    #uso_estq = uso_estq.rename(columns={'estq_max_01':'uso_estq_01'})

    for i in range(4, len(database)):
        if i < 9: #Somente para não ter erro com a coluna da direita        \/
            uso_estq[colunas[i]] = (retira_ae(database[colunas[i-1]]) - retira_ae(database[i][('estq_max_01')])) * percent
            uso_estq.columns = ['uso_estq'+prct+'_0'+str(i+1)]
            uso_estq = pd.merge(left=uso_estq, right=coluna, how='outer', on='MZUEUQRT')
        else:
            coluna = database[i][('estq_max_'+str(i+1))] - retira_ae(database[i][('estq_max_01')]) * percent
            coluna.columns = ['uso_estq'+prct+'_'+str(i+1)]
            uso_estq = pd.merge(left=uso_estq, right=coluna, how='outer', on='MZUEUQRT')
    
    pass

In [42]:
uso_estq.head(5)

,MZ,UEU,QRT,estq_max_01,estq_max_02,estq_max_03,estq_max_04,estq_max_05,estq_max_06,estq_max_07,estq_max_08,estq_max_09,estq_max_10,estq_max_11,estq_max_12,estq_max_13,estq_max_14,estq_max_15,estq_max_16
MZUEUQRT,,,,,,,,,,,,,,,,,,,
1004005,1.0,4.0,5.0,1388.5667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1151.5033,0.0,0.0,0.0,0.0
1004007,1.0,4.0,7.0,3136.8377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1190.6523,0.0,0.0,0.0,0.0
1004009,1.0,4.0,9.0,1892.4383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1835.2717,0.0,0.0,0.0,0.0
1004013,1.0,4.0,13.0,5340.5007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2454.1193,0.0,0.0,0.0,0.0
1004015,1.0,4.0,15.0,6983.3773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2330.1927,0.0,0.0,0.0,0.0


In [43]:
estq_max_num.head(5)

,MZ,UEU,QRT,estq_max_01,estq_max_02,estq_max_03,estq_max_04,estq_max_05,estq_max_06,estq_max_07,estq_max_08,estq_max_09,estq_max_10,estq_max_11,estq_max_12,estq_max_13,estq_max_14,estq_max_15,estq_max_16
MZUEUQRT,,,,,,,,,,,,,,,,,,,
1004005,1.0,4.0,5.0,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,1388.5667,2540.07,2540.07,2540.07,2540.07,2540.07
1004007,1.0,4.0,7.0,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,3136.8377,4327.49,4327.49,4327.49,4327.49,4327.49
1004009,1.0,4.0,9.0,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,1892.4383,3727.71,3727.71,3727.71,3727.71,3727.71
1004013,1.0,4.0,13.0,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,5340.5007,7794.62,7794.62,7794.62,7794.62,7794.62
1004015,1.0,4.0,15.0,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,6983.3773,9313.57,9313.57,9313.57,9313.57,9313.57
